#Lab.10 / IBM3202 – Cambios Conformacionales usando Modelos Basados en Estructura (SBM)

### Aspectos teóricos

La **teoría del paisaje energético** y el **principio de mínima frustración** en el plegamiento de proteínas han proporcionado la base teórica para la generación de modelos simplificados para simular las vías de plegamiento de diferentes proteínas. Es destacable que trabajos recientes han demostrado su utilidad para simular otros fenómenos funcionalmente relevantes, como el **mal plegamiento de proteínas y cambios conformacionales** asociados a la función biológica. La mayoría de estas aplicaciones se han generado mediante combinaciones astutas y cuidadosas de los términos enlazados y no enlazados nativos de **dos o más estructuras depositadas en el PDB** en dos o más estados conformacionales (es decir, conformaciones abiertas y cerradas, estados alfa y beta, etc.).


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_01.png'/>
<figcaption>FIGURA 1. Modelando las transiciones conformacionales de la adenilato quinasa. Esta enzima experimenta un movimiento de >25 Å entre estados abiertos (rojo, PDB 4AKE) y cerrados (verde, PDB 1AKE) debido a la unión de un ligando. La ocupación relativa de los estados cerrado y abierto puede ajustarse a datos experimentales variando la fuerza del subconjunto de contactos que solo existe en el estado cerrado entre 0.6 (rojo) y 1.2 (negro) en relación con los contactos abiertos.
 <br> Noel JK & Onuchic JN (2012) <i> Computational Modeling of Biological Systems, 31-54</i></figcaption></center>
</figure>

##Generalidades Experimentales

En este tutorial ejemplificaremos cómo podemos **combinar los contactos nativos de dos estructuras diferentes** para simular el cambio conformacional de la adenilato quinasa, una enzima que consta de tres dominios (LID, NMP y núcleo) y cataliza la reacción de fosforilación del AMP utilizando ATP, generando 2 moléculas de ADP como producto:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_02.png'/>
</center>
</figure>

Esta reacción requiere un cambio conformacional severo, el cual se puede observar en las estructuras de la proteína en presencia (1AKE) y en ausencia (4AKE) de sustratos.


#Part 0. Descarga e instalación del software necesario

Antes de comenzar, **recuerda iniciar el entorno de ejecución alojado** en Google Colab.

Luego, debemos instalar varios programas para realizar este tutorial, a saber:
- **biopython** para la manipulación de archivos PDB.
- **py3Dmol** para la visualización de la estructura de proteínas.
- **cpanm** para la instalación de varias utilidades Perl necesarias para ejecutar SMOG2.
- **SMOG2** para generar nuestros modelos basados en estructuras.
- **SBM-enhanced GROMACS** para preparar nuestro sistema de dinámica molecular (MD) y realizar simulaciones de MD.

Para visualizar nuestras trayectorias de MD, utilizaremos una versión web de **NGLview**. Esto se debe a la incapacidad de Google Colab para manejar un paquete de Python necesario para cargar NGLview directamente en Google Colab. Con suerte, esto cambiará en el futuro cercano.

1. Primero, comenzaremos descargando e instalando SMOG2 en Google Colab, lo cual requiere la instalación de varias utilidades Perl utilizando cpanminus.

**NOTA**: Esta instalación lleva aproximadamente ~10 minutos. Si es posible, realiza esta instalación antes de que comience la sesión del tutorial.


In [ ]:
# Descargando y extrayendo SMOG2 desde el servidor de SMOG
!wget http://smog-server.org/smog2/code/smog-2.4.2.tgz
!tar zxf smog-2.4.2.tgz

In [ ]:
# Configuración automática de cpan para Perl
!echo y | cpan
# Instalación de cpanm para facilitar la instalación de utilidades Perl
!cpan App::cpanminus
# Instalación de todas las utilidades Perl requeridas para SMOG2
!cpanm String::Util #--local-lib $nb_path
!cpanm XML::Simple #--local-lib $nb_path
!cpanm Exporter #--local-lib $nb_path
!cpanm PDL #--local-lib $nb_path
!cpanm XML::Validator::Schema #--local-lib $nb_path
# Nuevo módulo Perl agregado: requisito para smog v2.4.2
!cpanm XML::LibXML #--local-lib $nb_path

In [ ]:
# Descargar un archivo preconfigurado de SMOG2 y probar la instalación
%%bash
rm -r /content/smog-2.4.2/configure.smog2
wget -P /content/smog-2.4.2 https://github.com/pb3lab/ibm3202/raw/master/software/configure.smog2
source /content/smog-2.4.2/configure.smog2
smog2 -h

2. Luego, configuraremos nuestro GROMACS mejorado con SBM en Google Colab, basándonos en la versión previamente compilada e instalada de GROMACS.


In [ ]:
# Descargar y descomprimir la carpeta comprimida de GROMACS mejorado con SBM
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs_sbm.tar.gz
!tar xzf gromacs_sbm.tar.gz

In [ ]:
# Verificar si GROMACS mejorado con SBM está funcionando
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx help commands

3. Instalamos biopython y py3Dmol

In [ ]:
!pip install biopython py3dmol

Con esto instalado, estamos listos para empezar nuestros experimentos

# Parte I – Generar SBM de grano grueso usando SMOG2

Como hicimos en el tutorial anterior, primero descargaremos las coordenadas de las estructuras resueltas de la adenilato quinasa humana en los estados abierto (4AKE) y cerrado (1AKE) para generar SBMs con granulado grueso para ambas estructuras utilizando SMOG2. Luego, emplearemos una combinación de análisis numérico y SMOG2 para generar modelos SBM personalizados que contengan información de ambas estructuras en un solo archivo (**modelos de dos cuencas**).

1. Comenzamos creando y accediendo a una carpeta para preparar nuestros sistemas.


In [ ]:
# Creemos primero una carpeta. Necesitamos importar las bibliotecas os y path
import os
from pathlib import Path

# Luego, definimos la ruta de la carpeta que queremos crear.
# Ten en cuenta que la carpeta HOME para un entorno de ejecución alojado en colab es /content/
smogpath = Path("/content/prepare_dualAKE/")

# Ahora, creamos la carpeta utilizando el comando os.mkdir()
# La condición if simplemente verifica si la carpeta ya existe
# En ese caso, Python devuelve un error
if os.path.exists(smogpath):
  print("La ruta ya existe")
if not os.path.exists(smogpath):
  os.mkdir(smogpath)
  print("La ruta se creó correctamente")

In [ ]:
#Cambiar directorio
os.chdir(smogpath)

2. Luego, descargaremos las estructuras resueltas de la adenilato quinasa humana en las conformaciones abierta (PDB 4AKE) y cerrada (PDB 1AKE), y eliminaremos las conformaciones alternativas de las cadenas laterales, las moléculas de agua y los ligandos utilizando Biopython, como hemos hecho en tutoriales anteriores.

**NOTA:** Es posible que obtengas una advertencia de "discontinuidad de cadena" en Biopython. En este caso particular, esto se debe a la anotación no contigua de átomos no proteicos de las cadenas A y B en el archivo PDB.

In [ ]:
#Importando el PDB con biopython
import os
from Bio.PDB import *
pdbid = ['1ake', '4ake']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
# Aquí configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()

# Y aquí establecemos la conformación del residuo que queremos conservar
keepAltID = "A"
class KeepOneConfOnly(Select):  # Hereda métodos de la clase Select
    def accept_atom(self, atom):
        if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
            atom.set_altloc(" ")  # Elimina la identificación de ubicación alternativa antes de la salida.
            return True
        else:  # La ubicación alternativa no debe ser incluida en la salida.
            return False
        # fin de accept_atom()

# Y ahora iteramos para todas las estructuras
for s in pdbid:
  structure = parser.get_structure('X', s + "_cleaned.pdb")
  # Esto conservará solo la conformación para cada residuo
  io.set_structure(structure)
  io.save(s + "_ready.pdb", select=KeepOneConfOnly())
print("Tus PDBs fueron procesados. Se eliminaron las conformaciones alternativas de las cadenas laterales.")

In [ ]:
# Aquí configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()
for s in pdbid:
  structure = parser.get_structure('X', s + "_ready.pdb")
  # Y aquí eliminamos hidrógenos, aguas y ligandos usando Dice
  io.set_structure(structure)
  sel = Dice.ChainSelector('A', 1, 214)
  io.save(s + "_clean.txt", sel)
print("Tus PDBs fueron procesados. Solo se han conservado los átomos pesados de la proteína.")
print("Ambos PDBs se han guardado como archivos de texto para editar en Google Colab.")
print("Recuerda editarlos antes de usar SMOG2.")

3. Visualicemos las estructuras usando py3Dmol

In [ ]:
import py3Dmol
# Primero asignamos py3Dmol.view como un visor de dos paneles
view = py3Dmol.view(800, 400)
# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')
# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer la estructura del estado abierto
view.addModel(open('4ake_clean.txt', 'r').read(), 'pdb')
# Aquí configuramos el estilo y el color de la visualización
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'red'}})
# Aquí centramos la molécula para su visualización
view.zoomTo()
# Y finalmente visualizamos las estructuras con el comando a continuación
view.show()

In [ ]:
import py3Dmol
# Primero asignamos py3Dmol.view como un visor de dos paneles
view = py3Dmol.view(800, 400)
# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')
# Ahora hacemos lo mismo para la estructura del estado cerrado
view.addModel(open('1ake_clean.txt', 'r').read(), 'pdb')
# Aquí configuramos el estilo y el color de la visualización
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'green'}})
# Aquí centramos la molécula para su visualización
view.zoomTo()
# Y finalmente visualizamos las estructuras con el comando a continuación
view.show()

4. Como vimos en nuestro tutorial anterior, estos archivos PDB se guardan como archivos .txt para editarlos, según los requisitos de formato de SMOG2. **¡Corrige ambos archivos según sea necesario!**


5. Una vez corregidos, procesamos con SMOG2 como se indica abajo:

In [ ]:
%%bash
source /content/smog-2.4.2/configure.smog2
smog2 -i 4ake_clean.txt -CA -dname 4ake_smog
smog2 -i 1ake_clean.txt -CA -dname 1ake_smog

# Parte II – Generar un modelo personalizado de SBM de dos cuencas utilizando SMOG2


Ahora deberíamos tener nuestros modelos SBM para los estados abierto y cerrado de la adenilato quinasa humana. Sin embargo, si recuerdas de nuestras conferencias y nuestro tutorial anterior, estos modelos granulados gruesos no contienen moléculas de agua, ligandos, etc. Además, dado que no tenemos agua en nuestro sistema, la inclusión de un ligando llevará a que se desplace fuera del sitio activo hacia el infinito (como se puede ver en el archivo de simulación .mdp, no estamos utilizando condiciones de límite periódicas).

**¿Cómo vamos a simular un cambio conformacional en ausencia de ligandos?**

En lugar de pensar en incluir el ligando, debemos pensar en las **consecuencias de la unión al ligando**. En este caso, la unión del ligando hace que los tres dominios de la proteína se acerquen entre sí, lo que se traduce en que **se formen varios contactos nativos al unirse al ligando** (o que son exclusivos de la conformación cerrada).

En resumen, para simular el cambio conformacional de esta enzima, es más apropiado considerar que:
- El **estado abierto** es la **condición inicial**, ya que puede existir tanto en ausencia como a bajas concentraciones de ligando.
- Las interacciones nativas que son **exclusivas del estado cerrado** corresponden a interacciones **inducidas por el ligando**.
- Solo se deben incluir en un **mapa de contactos nativos combinados** (o potencial de doble cuenca) las **interacciones nativas inducidas por el ligando** que exhiben cambios significativos en la distancia (por ejemplo, dist [4AKE] / dist [1AKE] > 50%).

De hecho, esto se muestra en la siguiente figura de un artículo que inspiró este tutorial:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_03.jpg'/>
<figcaption>FIGURA 2. Gráfico de los contactos exclusivos de la forma cerrada de la adenilato quinasa. Cada punto representa un contacto entre el residuo <i>i</i> y el residuo <i>j</i> que es exclusivo de la forma cerrada. El eje Y es la distancia entre los átomos C${\alpha}$ de los residuos <i>i</i> y <i>j</i> en la forma abierta, y el eje X es la distancia en la forma cerrada. Los contactos por encima de la línea de pendiente 1.5 (línea sólida) constituyen el conjunto de contactos seleccionados para los modelos de SBM de doble cuenca.
<br> Whitford PC et al (2007) <i> J Mol Biol 366(5), 1661-1671</i></figcaption></center>
</figure>

1. **¿Cómo podemos filtrar primero los contactos que son exclusivos de la conformación cerrada?** Utilizaremos `grep` sobre los archivos **.contact.CG** que obtuviste durante el procesamiento de SMOG2 para los estados abierto y cerrado:

In [ ]:
# Creando una carpeta para preparar el modelo de SBM de doble cuenca
dualpath = Path("dualSBM")

# Ahora, creamos la carpeta utilizando el comando os.mkdir()
# La condición if simplemente verifica si la carpeta ya existe
# En ese caso, Python devuelve un error
if os.path.exists(dualpath):
  print("La ruta ya existe")
if not os.path.exists(dualpath):
  os.mkdir(dualpath)
  print("La ruta se creó correctamente")

# Cambiando a esta nueva carpeta
os.chdir(dualpath)

In [ ]:
# Utilizando grep para verificar qué líneas en el archivo1 (4ake) también están en el archivo2 (1ake)
# La bandera -v invierte el sentido de la coincidencia, imprimiendo así las líneas que no coinciden con el archivo2
!grep -Fxvf ../4ake_smog.contacts.CG ../1ake_smog.contacts.CG > uniquecontacts.txt

Si todo va bien, deberías obtener una lista de 113 contactos en el formato `cadena_i res_i cadena_j res_j`. Ahora, debemos evaluar si la distancia de estos contactos es significativamente diferente entre los estados abierto y cerrado.

2. Para determinar la diferencia de distancia entre estos contactos en los estados abierto y cerrado, haremos un pequeño truco. Primero, utilizaremos el módulo `trjconv` de GROMACS para generar un **archivo PDB granulado grueso** de ambos estados basado en nuestro archivo .gro granulado grueso de SBM. Luego, utilizaremos SMOG2, junto con estos archivos PDB granulado grueso y nuestra lista de contactos exclusivos para la estructura cerrada. Con este pequeño truco, obtendremos los **parámetros LJ (¡y por lo tanto las distancias!)** de los contactos inducidos por el ligando tanto en los estados abierto (4AKE) como cerrado (1AKE).

In [ ]:
# Generando archivos PDB granulado grueso para 4AKE y 1AKE usando GROMACS
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx editconf -f ../4ake_smog.gro -o 4ake_CA.pdb
gmx editconf -f ../1ake_smog.gro -o 1ake_CA.pdb
# Edita tu archivo para cumplir con los parámetros de SMOG2
grep ATOM 4ake_CA.pdb > 4ake_CA_clean.pdb
echo "END" >> 4ake_CA_clean.pdb
grep ATOM 1ake_CA.pdb > 1ake_CA_clean.pdb
echo "END" >> 1ake_CA_clean.pdb

In [ ]:
%%bash
source /content/smog-2.4.2/configure.smog2
smog2 -i 4ake_CA_clean.pdb -c uniquecontacts.txt -t /content/smog-2.4.2/share/templates/SBM_calpha -dname 4ake_unique
smog2 -i 1ake_CA_clean.pdb -c uniquecontacts.txt -t /content/smog-2.4.2/share/templates/SBM_calpha -dname 1ake_unique

Si recordamos de nuestro tutorial anterior, la sección `[ pairs ]` del archivo .top contiene los contactos nativos y sus parámetros.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_05.png'/>
</center>
</figure>

Dado que solicitamos el mismo mapa de contactos definido por el usuario para ambos archivos en el paso anterior, podemos obtener la diferencia de distancia entre cada contacto al **dividir la columna 4 (o 5) de un archivo por la columna 4 (o 5) de otro archivo**.

3. Create two text files on Google Colab in which you will store only the `[ pairs ]` section of these newly generated .top files. Then, we will use `awk` over these files to determine which interactions significantly change between these states:

In [ ]:
# Creando dos archivos que contienen solo las secciones de pares usando awk
# Y eliminando las primeras y últimas líneas impresas usando sed
!awk 'NR==1, $2 == "pairs" {next}; NF {print $0}; $2 == "exclusions" {exit}' 1ake_unique.top | sed '1d;$d' > closed.pairs
!awk 'NR==1, $2 == "pairs" {next}; NF {print $0}; $2 == "exclusions" {exit}' 4ake_unique.top | sed '1d;$d' > open.pairs

In [ ]:
!paste open.pairs closed.pairs | awk '{if(($4/$9)>1.5)print $1, $2, $3, $9, $10}' > Qligand.pairs

Después de esto, obtendremos un archivo de texto con **83 contactos** que son exclusivos del estado cerrado y para los cuales ocurre un cambio significativo en la distancia (>50%) al alcanzar la conformación abierta, no unida.

4. También usaremos `awk` para generar las listas de `[ exclusion ]` para estos contactos nativos inducidos por el ligando.

In [ ]:
!paste open.pairs closed.pairs | awk '{if(($4/$9)>1.5)print $1, $2}' > Qligand.exclusions

Ahora tenemos todo lo que necesitamos para generar un modelo de doble cuenca para simular el cambio conformacional de la adenilato quinasa humana: archivos de coordenadas y parámetros y contactos nativos inducidos por el ligando.

5. Tomaremos uno de los archivos .top para uno de los estados de la adenilato quinasa humana y agregaremos manualmente los pares y exclusiones inducidos por el ligando que acabamos de obtener. Dada nuestra suposición anterior sobre el cambio conformacional de esta proteína, **la estrategia más razonable es utilizar el archivo .top de la estructura abierta (4AKE) para su modificación.**

In [ ]:
# ¡Copia el archivo .top para 4AKE en esta carpeta y modifícalo!
!cp ../4ake_smog.top dualAKE.top

# Parte III – Ejecutar y analizar nuestras simulaciones de SBM de doble cuenca

Ahora, estamos listos para realizar nuestras simulaciones del cambio conformacional inducido por el ligando de la adenilato quinasa humana utilizando estos modelos SBM de doble cuenca.

1. Comenzaremos creando una nueva carpeta para preparar y ejecutar nuestras simulaciones de MD, en la cual copiaremos nuestros archivos de coordenadas y topología de SBM.


In [ ]:
# Definiendo una nueva carpeta para las simulaciones de MD
mdpath = Path("/content/md_dualAKE/")

# Ahora, creamos la carpeta utilizando el comando os.mkdir()
# La condición if simplemente verifica si la carpeta ya existe
# En ese caso, Python devuelve un error
if os.path.exists(mdpath):
  print("La ruta ya existe")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("La ruta se creó correctamente")

In [ ]:
# Cambiando al directorio recién creado y copiando los archivos .gro y .top
os.chdir(mdpath)
from shutil import copyfile
copyfile(smogpath/'4ake_smog.gro', mdpath/'dualAKE.gro')
copyfile(smogpath/dualpath/'dualAKE.top', mdpath/'dualAKE.top')

2. Luego, descargaremos el mismo **archivo de instrucciones de MD** que usamos en nuestro tutorial anterior (**mdrun_CA_v5.mdp**), cambiando la temperatura de simulación a 108 y el número de pasos a 5000000. También descargaremos el script Perl para generar nuestros potenciales tabulados de LJ 12-10.

In [ ]:
%%bash
wget https://github.com/pb3lab/ibm3202/raw/master/files/mdrun_CA_v5.mdp
wget https://github.com/pb3lab/ibm3202/raw/master/files/maketable4.pl
perl maketable4.pl > table.xvg

3. Por último, prepararemos nuestro **archivo .tpr de entrada de ejecución binaria portable para GROMACS** en esta carpeta y ¡ejecutaremos nuestra simulación! Ten en cuenta cómo indicamos a GROMACS que utilice nuestra tabla personalizada de potenciales tabulados de LJ 12-10.

Esta simulación tarda ~15 minutos.

In [ ]:
#Precompilando nuestro binario
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx grompp -f mdrun_CA_v5.mdp -c dualAKE.gro -p dualAKE.top -o run.tpr

In [ ]:
#Corriendo nuestra simulación
%%time
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx mdrun -s run.tpr -table table.xvg -tablep table.xvg -nt 2 -noddcheck

4. Una vez que nuestra simulación haya terminado, podemos analizar si se observa el cambio conformacional en nuestro archivo de trayectoria. Para simplificar, utilizaremos primero el módulo `rmsd` junto con la estructura inicial, abierta, como evidencia de este cambio.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Comandos de RMSD
echo "0" > options
echo " " >> options
echo "0" >> options
echo " " >> options
#Cálculo de RMSD
gmx rms -s dualAKE.gro -f traj_comp.xtc -xvg none < options

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('rmsd.xvg')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('rmsd (nm)')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

2. Una métrica mejor para observar el cambio conformacional sería determinar directamente si se forman los contactos nativos inducidos por el ligando. Para esto, usaremos nuevamente el módulo `g_kuh`, junto con la lista de contactos que son exclusivos de la estructura cerrada y la estructura cerrada como referencia para las distancias nativas. Por favor, verifica cómo estamos generando el archivo de mapa de contactos necesario para este análisis:


In [ ]:
# Generando el archivo de mapa de contactos para g_kuh
f = open(mdpath/"Qligand.ndx", "a")
f.write("[ Qligand ]\n")
with open(smogpath/dualpath/"Qligand.exclusions") as infile:
  for line in infile:
    if line.strip():
      cols = line.split()
      f.write(cols[0] + "\t" + cols[1] + "\n")
f.close()

# Copiando la estructura de referencia del estado cerrado para determinar las distancias nativas
copyfile(smogpath/'1ake_smog.gro', mdpath/'1ake_smog.gro')

In [ ]:
# Analizando la formación de contactos inducidos por el ligando en nuestra trayectoria
%%bash
source /content/gromacs_sbm/bin/GMXRC
g_kuh -s 1ake_smog.gro -f traj_comp.xtc -n Qligand.ndx -noabscut -noshortcut -cut 0.2

4. ¡Vamos a graficar nuestros resultados y ver qué sucedió durante nuestra simulación! Primero graficaremos el cambio en los contactos nativos (Q), y luego el cambio en la energía potencial. Puedes verificar el cambio en el RMSD tú mismo.


In [ ]:
!paste rmsd.xvg qvals.out > data.txt
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('data.txt')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Q')
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

5. Para finalizar, visualizaremos nuestra simulación. Para ello, utilizaremos el módulo `trjconv` para extraer solo la proteína de nuestro sistema y convertir nuestra trayectoria en un archivo PDB. Luego, descargaremos este nuevo archivo PDB y lo cargaremos en [**NGLviewer**](http://nglviewer.org/ngl/) como un archivo PDB de **trayectoria**.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Truco para dar opción a gmx interactivo
echo "Protein" > options
echo "Protein" >> options
echo " "
gmx trjconv -s run.tpr -f traj_comp.xtc -o traj.pdb < options

In [ ]:
#Descarga de la trayectoria
from google.colab import files
files.download("/content/md_dualAKE/traj.pdb")

**Y llegamos al final del décimo tutorial!** Buena ciencia!